In [36]:
import pandas as pd
import openpyxl
import awswrangler as awr

pegando excel

In [37]:
excel_path = r"C:\Users\raphael.almeida\Documents\Processos\relatorio_inadimplencia\historico_faturas.xlsx"

df_base = pd.read_excel(excel_path, engine='openpyxl')



ajustes pontuais base

In [38]:
colunas_comuns = [
    "matricula",
    "conjunto",
    "cooperativa",
    "ponteiro",
    "numero_boleto",
    "nosso_numero",
    "unidade",
    "associado",
    "data_vencimento",
    "data_baixa",
    "valor_titulo",
    "valor_baixa",
    "data_emissao",
    "data_atualizacao",
    "situacao"
]

df_base = df_base[colunas_comuns]
df_base = df_base.drop_duplicates(subset=['matricula', 'conjunto', 'cooperativa','ponteiro', 'situacao', 'data_atualizacao'])

df_base['data_vencimento'] = pd.to_datetime(df_base['data_vencimento'])
df_base['data_vencimento'] = df_base['data_vencimento'].dt.date 

df_base['data_baixa'] = pd.to_datetime(df_base['data_baixa'])
df_base['data_baixa'] = df_base['data_baixa'].dt.date 

df_base['data_emissao'] = pd.to_datetime(df_base['data_emissao'])
df_base['data_emissao'] = df_base['data_emissao'].dt.date 

tratando colunas nulas

In [39]:
df_base['data_baixa'] = df_base['data_baixa'].fillna(pd.Timestamp('1900-01-01'))
df_base['data_emissao'] = df_base['data_emissao'].fillna(pd.Timestamp('1900-01-01'))
df_base['data_vencimento'] = df_base['data_vencimento'].fillna(pd.Timestamp('1900-01-01'))
df_base['valor_titulo'] = df_base['valor_titulo'].fillna(0)
df_base['valor_baixa'] = df_base['valor_baixa'].fillna(0)
df_base['nosso_numero'] = df_base['nosso_numero'].fillna('NULL')
df_base['numero_boleto'] = df_base['numero_boleto'].fillna('NULL')

gerando base de inadimplência

In [40]:
query_path = r"C:\Users\raphael.almeida\Documents\Processos\relatorio_inadimplencia\sql\faturas_inadimplentes.sql"

with open(query_path, 'r') as file:
    query = file.read()

df_inadimplentes = awr.athena.read_sql_query(query, database='silver')
df_inadimplentes = df_inadimplentes.drop_duplicates('ponteiro', keep='first')

df_inadimplentes.loc[:, 'data_atualizacao'] = df_inadimplentes['data_vencimento']
df_inadimplentes.loc[:, 'situacao']='INADIMPLENTE'


selecionando limite base

In [41]:
today = pd.to_datetime('today').date()
year = today.year
month = today.month
df_inadimplentes['data_vencimento'] = pd.to_datetime(df_inadimplentes['data_vencimento'])


df_inadimplentes = df_inadimplentes[df_inadimplentes['data_vencimento']> pd.to_datetime('2025-05-31')]
#df_inadimplentes = df_inadimplentes[df_inadimplentes['data_vencimento'].dt.month == month]

#df_inadimplentes = df_inadimplentes[df_inadimplentes['data_vencimento'].dt.year == year]




tratando colunas df_inadimplentes

In [42]:

for col in colunas_comuns:
    if col not in df_inadimplentes.columns:
        df_inadimplentes.loc[:, col] = pd.NA

df_inadimplentes = df_inadimplentes[colunas_comuns]

concatenando e removendo linhas inteiras duplicadas

In [43]:
df_composto_inadimplentes =  pd.concat([df_base, df_inadimplentes])
df_composto_inadimplentes = df_composto_inadimplentes.drop_duplicates(keep='first')


C:\Users\raphael.almeida\AppData\Local\Temp\ipykernel_14036\3111717009.py:1: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_composto_inadimplentes =  pd.concat([df_base, df_inadimplentes])


filtrando apenas os ponteiros de inadimplentes

In [44]:
#df_composto_inadimplentes['data_vencimento'] = pd.to_datetime(df_composto_inadimplentes['data_vencimento'])
#df_composto_inadimplentes_mes = df_composto_inadimplentes[df_composto_inadimplentes['data_vencimento'].dt.month == month]
#df_composto_inadimplentes_mes = df_composto_inadimplentes[df_composto_inadimplentes['data_vencimento'].dt.year == year]

df_inadimplentes_lista = df_composto_inadimplentes.loc[df_composto_inadimplentes['data_vencimento'].notna() ,'ponteiro'].to_list()


gerando o relatório de pagamentos do mês e tratando colunas

In [45]:
query_path = r"C:\Users\raphael.almeida\Documents\Processos\relatorio_inadimplencia\sql\faturas_baixadas.sql"

with open(query_path, 'r') as file:
    query = file.read()

df_pagamentos = awr.athena.read_sql_query(query, database='silver')


for col in colunas_comuns:
    if col not in df_pagamentos.columns:
        df_pagamentos.loc[:, col] = pd.NA
df_pagamentos = df_pagamentos[colunas_comuns]

tratando colunas

In [46]:
df_pagamentos.loc[:, 'data_atualizacao'] = df_pagamentos['data_baixa']
df_pagamentos.loc[:, 'situacao']='PAGO'

In [47]:


df_pagamentos['data_baixa'] = pd.to_datetime(df_pagamentos['data_baixa'])
df_pagamentos['data_vencimento'] = pd.to_datetime(df_pagamentos['data_vencimento'])

df_pagamentos = df_pagamentos.drop_duplicates('ponteiro', keep='first')
df_pagamentos = df_pagamentos[
    (df_pagamentos['data_baixa']> pd.to_datetime('2025-05-31'))&
    (df_pagamentos['data_vencimento']> pd.to_datetime('2025-05-31'))
    ]
#df_pagamentos = df_pagamentos[df_pagamentos['data_baixa'].dt.month == month]
#df_pagamentos = df_pagamentos[df_pagamentos['data_baixa'].dt.year == year]

df_pagamentos_inadimplencia = df_pagamentos[df_pagamentos['ponteiro'].isin(df_inadimplentes_lista)]


concatenando faturas baixadas dos vencidos deste mes com a base composta e atualizada

In [48]:
df_atualizado = pd.concat([df_composto_inadimplentes, df_pagamentos_inadimplencia])
df_atualizado.drop_duplicates(subset=['matricula', 'conjunto', 'cooperativa', 'ponteiro', 'situacao', 'data_atualizacao'])

C:\Users\raphael.almeida\AppData\Local\Temp\ipykernel_14036\3245592541.py:1: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_atualizado = pd.concat([df_composto_inadimplentes, df_pagamentos_inadimplencia])


,matricula,conjunto,cooperativa,ponteiro,numero_boleto,nosso_numero,unidade,associado,data_vencimento,data_baixa,valor_titulo,valor_baixa,data_emissao,data_atualizacao,situacao
0,20400,10238,Stcoop,157214,1135680,1135680-1,UNIDADE CASCAVEL,VOLMAR DE ALMEIDA,2025-06-05,1900-01-01,1032.33,0.00,2024-08-05,2025-06-05 00:00:00,INADIMPLENTE
1,17883,10354,Stcoop,159226,1137286,1137286-6,UNIDADE ITAJAI,VALÉRIO JOSÉ SEBASTIÃO,2025-06-10,1900-01-01,995.52,0.00,2024-08-12,2025-06-10 00:00:00,INADIMPLENTE
2,4322,14269,Stcoop,225329,509,37422190000000509,UNIDADE ITAPEJARA,MAICON ABATI,2025-06-10,1900-01-01,952.30,0.00,2025-04-25,2025-06-10 00:00:00,INADIMPLENTE
3,14562,12199,Stcoop,188991,1161436,1161436-9,UNIDADE ITAPEJARA,G B K - TRANSPORTES LTDA,2025-06-10,1900-01-01,2125.65,0.00,2024-11-06,2025-06-10 00:00:00,INADIMPLENTE
4,5598,10992,Stcoop,168777,1145535,1145535-5,UNIDADE ITAPEJARA,PAULO ROBERTO RABELO,2025-06-10,1900-01-01,975.63,0.00,2024-09-09,2025-06-10 00:00:00,INADIMPLENTE
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
57723,1735,3725,Viavante,173201,1072511,1072511-0,UNIDADE UBERLÂNDIA,MJ POWER TRANSPORTES LTDA,2025-06-05 00:00:00,2025-06-05,NaN,2655.28,NaN,2025-06-05,PAGO
89511,589,4764,Viavante,219782,1115862,1115862-9,UNIDADE MARINGÁ,HIGO TRANSPORTES RODOVIARIO LTDA,2025-06-05 00:00:00,2025-06-05,NaN,1277.57,NaN,2025-06-05,PAGO
102741,250,1412,Viavante,226204,1121538,1121538-7,UNIDADE ITAPEJARA,JEAN WEBER VENDRUSCOLO E CIA LTDA,2025-06-05 00:00:00,2025-06-02,NaN,2738.69,NaN,2025-06-02,PAGO
153808,297,3645,Viavante,169287,1068841,1068841-4,UNIDADE ITAJAI,MUNIZ TRANSPORTES E SERVICOS LTDA,2025-06-05 00:00:00,2025-06-05,NaN,857.13,NaN,2025-06-05,PAGO


ajustando formatação

In [49]:

df_atualizado['data_vencimento'] = pd.to_datetime(df_atualizado['data_vencimento'])
df_atualizado['data_vencimento'] = df_atualizado['data_vencimento'].dt.date 

df_atualizado['data_baixa'] = pd.to_datetime(df_atualizado['data_baixa'])
df_atualizado['data_baixa'] = df_atualizado['data_baixa'].dt.date 

df_atualizado['data_emissao'] = pd.to_datetime(df_atualizado['data_emissao'])
df_atualizado['data_emissao'] = df_atualizado['data_emissao'].dt.date 

df_atualizado['data_atualizacao'] = pd.to_datetime(df_atualizado['data_atualizacao']).dt.date

df_atualizado = df_atualizado[colunas_comuns]

tratando colunas nulas

In [50]:
df_atualizado['data_baixa'] = df_atualizado['data_baixa'].fillna(pd.Timestamp('1900-01-01'))
df_atualizado['data_vencimento'] = df_atualizado['data_vencimento'].fillna(pd.Timestamp('1900-01-01'))
df_atualizado['valor_titulo'] = df_atualizado['valor_titulo'].fillna(0)
df_atualizado['valor_baixa'] = df_atualizado['valor_baixa'].fillna(0)
df_atualizado['nosso_numero'] = df_atualizado['nosso_numero'].fillna('NULL')
df_atualizado['numero_boleto'] = df_atualizado['numero_boleto'].fillna('NULL')

retornando ao excel base atualizado

In [51]:
excel_save_path = r"C:\Users\raphael.almeida\Documents\Processos\relatorio_inadimplencia\historico_faturas.xlsx"
df_atualizado.to_excel(excel_save_path, engine='openpyxl', index=False, sheet_name='historico_faturas')